# Epigenetic score
A penalised regression model with the human Dex residuals as the outcome and the 496 cross-tissue CpGs as predictors was run using the glmnet package in R. 

In [1]:
library(glmnet)
library(corrplot)

Loading required package: Matrix
Loaded glmnet 3.0

corrplot 0.84 loaded


In [4]:
# human methylation data of MPIP cohort with dex residualized by sex, case/control status, age, bmi and cell counts.
data <- readRDS("data/DNAm_of_overlapping_hpc_and_human.Rds")
data[1:10,1:5]

cg17001566,cg26786407,cg23370548,cg23987789,cg21774136
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.2175461,0.6984534,0.6878971,0.1397572,0.5187689
0.2507799,0.7036661,0.6863872,0.1530025,0.5416056
0.1632386,0.6433354,0.6373376,0.1374984,0.4258133
0.1138560,0.6895280,0.7174286,0.1244146,0.4514850
0.2506501,0.6054890,0.6623528,0.2008409,0.4481249
0.2011912,0.6591903,0.6722256,0.1399751,0.5245547
0.2944039,0.6334535,0.6791212,0.1955723,0.4883494
0.2741708,0.6517067,0.6500248,0.2073143,0.4897958
0.3621200,0.7008398,0.7288209,0.1652263,0.5595763


In [6]:
# determine EN alpha

#corrrelation strengh c(x)
meth = data[,-grep("residuals.dex.human", colnames(data))]
cor_mat=cor(meth[,-c(dim(meth)[2]-1,dim(meth)[2])])
#get upper triangle matrix
tmp=cor_mat
tmp[row(cor_mat)>col(cor_mat)] = 0
diag(tmp) = 0
#Frobenius norm 
Fnorm = norm(tmp, type="F")
c = Fnorm / sqrt( (dim(cor_mat)[1]^2-dim(cor_mat)[1]) / 2)
# get alpha
alpha= 10^(-c)
alpha

[1] 0.2761701

In [7]:
# The penalised regression model was run 100 times and the best-fit lambda values were extracted. 
write.table(cbind("lambda.min","lambda.1se" ), file="data/lambda_selection_100_elastic_long.txt", quote = F,sep="\t", col.names = F, row.names = F)
for(i in 1:100){
  x= as.matrix(na.omit(data)[,-grep("residuals.dex.human", colnames(data))]) #exlude residuals
  y=na.omit(data$residuals.dex.human)
  alpha_value=alpha #is the lasso penalty,
  cvfit = cv.glmnet(x, y, alpha = alpha_value, standardize = T )
  write.table(cbind(cvfit$lambda.min,cvfit$lambda.1se ), file="data/lambda_selection_100_elastic_long.txt", append = T, quote = F,sep="\t", col.names = F, row.names = F)
}  

In [10]:
# Default settings for the cross-validation glmnet model were considered: 10-fold cross validation and avg. lambda from the 100 runs to give the optimal solution.
lambda = read.delim("data/lambda_selection_100_elastic_long.txt", head=T)
cvfit = cv.glmnet(x, y, alpha = alpha, standardize = T )
coef.fit=coef(cvfit, s=mean(lambda$lambda.1se))
index.fit <- which(coef.fit[,1] !=0) 
variables<-row.names(coef.fit)[index.fit]
variables<-variables[ !(variables %in% '(Intercept)')]
coef.value<-coef.fit[index.fit,]
coef.value[-1]
#write.table(coef.value[-1], "data/EpiScore_Cpgs_n24.txt", row.names = T,quote = F,col.names = F)

cg02862467   cg02927682   cg07052737   cg01400750   cg03637218   cg16141752 
-0.165623312 -0.108305162 -0.084929249 -0.460991378 -0.080656093 -0.262948270 
  cg11671363   cg20977312   cg09571972   cg16312968   cg13718827   cg11316887 
-0.269960338 -0.175183210 -0.104389445 -0.431250409 -0.675091747  0.002167950 
  cg21344746   cg13581015   cg24850296   cg18020065   cg04958055   cg12157761 
-0.215960188 -0.031250374 -0.060322385 -0.010992178 -0.371285125 -0.259598140 
  cg03543954   cg08297985   cg04674060   cg23987336   cg07830557   cg19118951 
-0.103462545 -0.195981668  0.389974414 -0.192186786 -0.055120591 -0.004694544